In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
transform  = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

batch_size = 64
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform= transform)
trainload = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
testload = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
classes=('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
features = [64,64,'M',128,128,'M',256,256,256,256,'M',512,512,512,512,'M',512,512,512,512,'M']

class VGG19(nn.Module):
  def __init__(self):
    super(VGG19,self).__init__()
    self.make_layers = self.Convlayers()
    self.max_pool = nn.AdaptiveMaxPool2d((1, 1))
    self.fcs = nn.Sequential(
                              nn.Linear(in_features=512, out_features=4096, bias=True),
                              nn.ReLU(inplace=True),
                              nn.Dropout(p=0.5),
                              nn.Linear(in_features=4096, out_features=4096, bias=True),
                              nn.ReLU(inplace=True),
                              nn.Dropout(p=0.5),
                              nn.Linear(in_features=4096, out_features=10, bias=True),
                      
                           )

  def Convlayers(self):
    Conv=[]
    in_channels=3
    for x in features:
      if(x!='M'):
        Conv += [nn.Conv2d(in_channels=in_channels, out_channels=x, kernel_size=3, stride=1, padding=1) , nn.BatchNorm2d(x), nn.ReLU(inplace=True)]
        in_channels=x
      else:
        Conv += [nn.MaxPool2d(kernel_size=2, stride=2)]
    return nn.Sequential(*Conv)

  def forward(self,x):
    x=self.make_layers(x)
    x=self.max_pool(x)
    x = torch.flatten(x, start_dim = 1, end_dim = -1)
    x = self.fcs(x)
    return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

lr = 3e-4

model = VGG19().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

for epoch in range(10):
  for image, label in trainload:
    image = image.to(device)
    label = label.to(device)
    output = model(image)
    loss = loss_function(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print("Epoch", epoch + 1, "done")

print("Finished training")

Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done
Epoch 6 done
Epoch 7 done
Epoch 8 done
Epoch 9 done
Epoch 10 done
Finished training


In [ ]:

with torch.no_grad():
  corr=0
  incorr=0
  for image,label in testload:
    image = image.to(device)
    label = label.to(device)
    output = model(image)
    temp, pred = torch.max(output,1)
    corr += (pred==label).sum().item()
    incorr += (pred != label).sum().item()
  errorpercent = (corr/(corr+incorr))*100
  print("accuracy of model is ",errorpercent,"%")


accuracy of model is  82.76 %
